# CNTL temperature dynamic system

## Import package

In [1]:
import numpy as np
import joblib as jl

from matplotlib import pyplot as plt

## Load data

In [2]:
case  = 'CNTL'
fname = f'/home/b11209013/2024_Research/MPAS/LRF/Tseries_file/{case}.joblib'

data = jl.load(fname)
tpc1 = data['pc1']['t']
tpc2 = data['pc2']['t'] # unit in K

# time 
time = np.linspace(0, 32, 33)*0.25 # unit in day

## Compute finite different of the PCs

### Compute the derivartive of the PC time series

In [3]:
tpc1_dt = np.gradient(tpc1, time, edge_order=2) # unit in K/day
tpc2_dt = np.gradient(tpc2, time, edge_order=2)

## Constructing dynamic system

In [4]:
t_tend = np.stack([tpc1_dt, tpc2_dt], axis=0)
t_vec  = np.stack([tpc1, tpc2], axis=0)

M  = (t_tend @ t_vec.T) @ np.linalg.inv(t_vec @ t_vec.T) # unit in 1/day

print(M)

[[-0.66922963 -1.26279958]
 [ 0.82966373  0.68519988]]


## Check the correctness of the growth rate

In [5]:
sigma = jl.load(f'sigma/{case}.joblib')

wn = np.linspace(-359, 360, 720)
fr = np.linspace(1/30, 2, 60)

wnm, frm = np.meshgrid(wn, fr)

kel_curves = lambda ed, k: (86400/(2*np.pi*6.371e6))*np.sqrt(9.81*ed)*k

cond = (
    (wnm >= 1) & (wnm < 14) &
    (frm >= 1/20) & (frm <= 1/2.5) &
    (frm <= kel_curves(90, wnm)) & (frm >= kel_curves(8, wnm))
)

sigma_filted = {
    'pc1': np.where(cond, sigma['pc1'], np.nan),
    'pc2': np.where(cond, sigma['pc2'], np.nan)
}
print('Averaged Growth Rate of the CCKWs at M1 over Filtered: ', np.nanmean(sigma_filted['pc1']))
print('Averaged Growth Rate of the CCKWs at M2 over Filtered: ', np.nanmean(sigma_filted['pc2']))

Averaged Growth Rate of the CCKWs at M1 over Filtered:  -8.2764703206269
Averaged Growth Rate of the CCKWs at M2 over Filtered:  0.30771123060855854


## Computing e-folding time scale

In [6]:
eigvals, eigvec = np.linalg.eig(M)
print('Eigenvalue of the LIM: \n', eigvals)
print('Eigenvectors of the LIM: \n', eigvec)

# growth/decay time
gd_time = 1/np.real(eigvals)
period  = 2*np.pi / np.imag(eigvals)
print('\n')
print('Growth/Decay time of CCKW: ', gd_time[0])
print('Period of CCKW: ', period[0])

Eigenvalue of the LIM: 
 [0.00798512+0.76751495j 0.00798512-0.76751495j]
Eigenvectors of the LIM: 
 [[ 0.776852  +0.j          0.776852  -0.j        ]
 [-0.41661056-0.47216164j -0.41661056+0.47216164j]]


Growth/Decay time of CCKW:  125.23285500394046
Period of CCKW:  8.186401244430467
